# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a membrane defined by a mesh

In [ ]:
from bioexplorer import BioExplorer, MovieMaker, Cell, Membrane, Protein, AnimationParams, Vector2, Vector3, Quaternion
import glob

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
obj_folder = resource_folder + 'obj/'
membrane_folder = pdb_folder + 'membrane/'
lipids_folder = membrane_folder + 'lipids/'
transporters_folder = pdb_folder + 'transporters/'

### Configuration

In [ ]:
mesh_source= obj_folder + 'suzanne.obj'
scale = Vector3(2.5, 2.5, 2.5)

protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_atom_radius_multiplier = 1.0

## Camera position

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.114, 0.341, 0.020, 0.932],
    position=[368.152, 107.242, 367.312],
    target=[50.364, -6.824, -5.159]
)

### Membrane

In [ ]:
def add_membrane(
        frame, add_receptors,
        position=Vector3(0.0, 0.0, 0.0), 
        rotation=Quaternion(1.0, 0.0, 0.0, 0.0)):
    
    name = 'Suzanne'
    clip_planes = [[-1.0, 0.0, 0.0, 50.0]]

    # ACE2 receptor definition
    ace2_receptor = Protein(
        name=be.NAME_TRANS_MEMBRANE + '_ACE2',
        source=pdb_folder + '6m18.pdb', 
        transmembrane_params=Vector2(1.0, 2.0),
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        animation_params=AnimationParams(1, frame * 10 + 1, 1.0, frame * 10 + 2, 0.025),
        occurences=20)

    # GLUT3 definition
    transporter = Protein(
        name=be.NAME_TRANS_MEMBRANE + '_GLUT3',
        source=transporters_folder + '4zwc.pdb',
        transmembrane_params=Vector2(1.0, 2.0),
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        animation_params=AnimationParams(2, frame * 10 + 3, 1.0, frame * 10 + 4, 0.025),
        chain_ids=[1], occurences=30)

    # Membrane definition
    pdb_lipids = glob.glob(lipids_folder + '*.pdb')[:8]

    membrane = Membrane(
        lipid_sources=pdb_lipids, lipid_density=1.0,
        load_non_polymer_chemicals=True, load_bonds=True,
        animation_params=AnimationParams(3, frame * 10 + 5, 1.0, frame * 10 + 6, 0.025)
    )

    # Cell definition
    cell = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_MESH,
        shape_params=scale,
        shape_mesh_source=mesh_source, 
        membrane=membrane,
        proteins=[ace2_receptor, transporter]
    )

    # Add cell to scene
    status = be.add_cell(
        cell=cell, representation=protein_representation,
        position=position, rotation=rotation, clipping_planes=clip_planes)

    return status

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

## Animation

In [ ]:
from IPython.display import clear_output

mm = MovieMaker(be)
output_folder = '/tmp'

pos = Vector3(0.0, 0.0, 0.0)
rot_start = Quaternion(1.0, 0.0, 0.0, 0.0)
rot_end = Quaternion(0.0, 0.0, 1.0, 0.0)

nb_frames = 36

for frame in range(0, nb_frames):
    clear_output()
    be.reset_scene()
    be.set_general_settings(model_visibility_on_creation=False)
    print('Frame %d' % frame)
    rot = Quaternion.slerp(rot_start, rot_end, float(frame) / nb_frames)
    add_membrane(
        frame=frame, add_receptors=True,
        position=pos, rotation=rot)
    be.apply_default_color_scheme(be.SHADING_MODE_NONE)
    be.set_models_visibility(True)

    '''Snapshot'''
    mm.create_snapshot(
        renderer='bio_explorer',
        size=[512, 512], samples_per_pixel=16,
        path=output_folder,
        base_name='%05d' % frame)